In [1]:
import pandas as pd
import os
import tabula

In [2]:
# Функция для очистки сырой таблицы

def clean(data):
    
    # Очистка таблицы от лишних значений в столбце с регионами, удаление полностью пустых столбцов    
    clean_data = data[data[0].str.lower() != 'республика, край, область']
    clean_data = clean_data[~clean_data[0].str.startswith('*')].dropna(subset=[0]).dropna(axis=1, how='all')

    # Если регионы склеились с первым столбцом значений, разбиваем их
    if clean_data[0].str.contains(r'\d').any():
        region = clean_data[0].str.replace(r'\d+', '', regex=True).str.rstrip() 
        missed_data = clean_data[0].str.extractall(r'(\d+)')
        missed_data = missed_data.groupby(level=0)[0].apply(lambda x: ''.join(x))
        clean_data = clean_data.drop(columns=0)
        clean_data.insert(0, '0', region)
        clean_data.insert(1, 'missed_data', missed_data)    
        
    #Разбиваем столбцы со значениями, которые не должны были склеиваться - по дефисам или по пробелам  
    for col in clean_data.columns[1:]:
        if (clean_data[col].str.contains('--', na=False).any()) and (~clean_data[col].str.contains(r'\d', na=False).any()):
            split_data = clean_data[col].str.split('-', expand=True).drop(columns = 0).replace('', '-')
            split_data.columns = range(split_data.shape[1])
        else:
            split_data = clean_data[col].str.split(r'\s+', expand=True)
        for new_col in range(split_data.shape[1]):
            clean_data[f'{col}_{new_col}'] = split_data[new_col]
        clean_data.drop(columns = col, inplace=True)
        
   # Склеиваем столбцы, которые в источнике были одним
    def combine_columns(data):
        cols = data.columns[1:]
        i = 0
        while i < len(cols) - 1:
            if (data[cols[i]].isnull() ^ data[cols[i+1]].isnull()).all():
                data[cols[i]] = data[cols[i]].combine_first(data[cols[i+1]])
                data.drop(columns=cols[i+1], inplace=True)
                cols = data.columns
            else:
                i += 1
        return data
    clean_data = combine_columns(clean_data)
    clean_data.columns = range(clean_data.shape[1])
    return clean_data

# Функция для обработки названий регионов
def only_lower_letters(column):
    def inner_func(s):
        return ''.join([str.lower(char) for char in s if char.isalpha()])  
    return [inner_func(s) for s in column]

1. Подгружаем справочники (indicators, nosologies, years, regions) и приводим их в удобный вид.

In [3]:
indicators = pd.read_csv('data/raw/appendix/indicators.csv', encoding='windows-1251', sep=';')

In [4]:
nosologies = pd.read_csv('data/raw/appendix/nosologies.csv', encoding='windows-1251', sep=';')
nosologies = pd.melt(nosologies, \
                     id_vars=nosologies.columns[:5], \
                     value_vars=nosologies.columns[5:], \
                     var_name='year', value_name='table_number', ignore_index=False)
nosologies = nosologies.dropna(subset=['table_number']) \
                    .astype({'year': 'int', 'table_number': 'int'}) \
                    .sort_values(['year', 'table_number'])
                        
# Создаем столбец, показывающий номер первой части таблицы по порядку расположения таблиц внутри сборника
table_order = []
for y in nosologies.year.unique():
    x=0
    for p_q in nosologies.query('year == @y').pages_quantity:
        table_order.append(x)
        x += p_q
nosologies['table_order'] = table_order

# Обрабатываем комментарии, чтобы позже подтянуть их только к определенным индикаторам
nosologies = nosologies.merge(nosologies.comment.str.split(';indicator_short_code = ', expand=True), \
                                left_index=True, right_index=True) \
                        .drop(columns='comment') \
                        .drop_duplicates() \
                        .rename(columns={0: 'comment', 1: 'comment_for_indicator'})

In [5]:
years = pd.read_csv('data/raw/appendix/years.csv', encoding='windows-1251', sep=';')

In [6]:
regions = pd.read_csv('https://raw.githubusercontent.com/tochno-st/regions_dict/main/regions_etalon_v2.0.csv', \
                      sep=';', dtype='string')
regions = regions[['name_rus', 'okato', 'oktmo', 'level']]

regions['region'] = only_lower_letters(regions.name_rus)
regions = regions.rename(columns = {'name_rus': 'object_name', \
                                    'okato': 'object_okato', \
                                    'oktmo': 'object_oktmo', \
                                    'level': 'object_level'})

2. Обрабатываем сборники.

In [7]:
# Собираем пути сборников
folder = "data/raw/digests/"
paths = [folder + i for i in os.listdir(folder)]

In [8]:
# Создаем датафрейм для данных из всех сборников
all_data_pdf = pd.DataFrame()

# Для каждого сборника из папки:
for file_path in paths: 
    
    # 1. Достать год из названия файла.
    year_pdf = int(file_path[-8:-4])
    
    # 2. В соответствии с годом достать страницы из справочника years.
    pages_pdf = years.query('year == @year_pdf').pages.iloc[0] 
    
    # 3. Считать необходимые страницы из pdf.
    raw_data_pdf = tabula.read_pdf(file_path, pages=pages_pdf, stream=True, area=[162,0,813,595], \
                                   pandas_options={'header': None, 'dtype': 'string'})
    
    # 4. Собрать список номеров первых частей таблиц.
    first_tables = nosologies.query('year == @year_pdf').table_order.to_list()
   
    # 5. Создать датафрейм для сборника.
    year_data_pdf = pd.DataFrame()
    
    # 6. Обработать считанные таблицы в сборнике.
    # Для каждой первой части таблицы:
    for i in first_tables:
        
        # - прогнать ее через функцию clean
        # - соединить ее с частями на следующих страницах (всего бывает 2 части или 4 - см. pages_quantity)
        table = pd.concat([clean(raw_data_pdf[i]), clean(raw_data_pdf[i+1])], ignore_index=True)
        if nosologies.query('year == @year_pdf & table_order == @i').pages_quantity.iloc[0] == 4:
            table = table.merge(pd.concat([clean(raw_data_pdf[i+2]), clean(raw_data_pdf[i+3])], ignore_index=True), \
                                how='left', on=0)
        # - проставить соответствующие коды колонок-индикаторов
        table.columns = range(table.shape[1])
        table_data_pdf = pd.DataFrame(table)
        col = years.query('year == @year_pdf').indicators_order.iloc[0].split(';')
        col.insert(0, 'region')
        table_data_pdf.columns = col[:table.shape[1]]
        
        # - сделать анпивот таблицы
        table_data_pdf = pd.melt(table_data_pdf, id_vars=table.columns[0], value_vars=table.columns[1:], \
                                    var_name='indicator_short_code', value_name='indicator_value', ignore_index=False)
        
        # - добавить столбцы с соответствующими данными из приложения nosologies
        nosologies_data = nosologies.query('year == @year_pdf & table_order == @i') \
                                [['nosology_name','nosology_code','soc_dem_group', 'comment', 'comment_for_indicator']].iloc[0]
        table_data_pdf[['nosology_name','nosology_code','soc_dem_group','comment','comment_for_indicator']] = nosologies_data
        table_data_pdf.comment = [comment if indicator == comment_for_indicator else None
                for indicator, comment, comment_for_indicator in 
                zip(table_data_pdf['indicator_short_code'], table_data_pdf['comment'], table_data_pdf['comment_for_indicator'])]
        table_data_pdf = table_data_pdf.drop(columns='comment_for_indicator')
        
        # - добавить полученную таблицу в общий датафрейм для сборника
        year_data_pdf = pd.concat([year_data_pdf, table_data_pdf], ignore_index=True)
        
    # Всем таблицам проставить столбец с годом текущего сборника.
    year_data_pdf[['year']] = year_pdf
    
    # Добавить данные по сборнику в общий датафрейм со всеми сборниками
    all_data_pdf = pd.concat([all_data_pdf, year_data_pdf], ignore_index=True)

3. Приводим данные в нужный формат и дополняем данными из справочников.

In [9]:
# Заменяем пропуски в значениях индикаторов, округляем значения до 4 знаков после запятой
all_data_pdf.indicator_value = all_data_pdf.indicator_value.str.replace(',', '.')

mask = (all_data_pdf.indicator_value.isnull()) & (all_data_pdf.year == 2011)
all_data_pdf.loc[mask, 'indicator_value'] = '66666666'

all_data_pdf.indicator_value = pd.to_numeric(all_data_pdf.indicator_value, errors='coerce').fillna(88888888).round(4)

all_data_pdf.indicator_short_code = all_data_pdf.indicator_short_code.astype('int')

In [10]:
# Приводим регионы в соответствие со справочником
all_data_pdf.region = all_data_pdf.region.str.replace('г. ', '', regex=False)
all_data_pdf.region = only_lower_letters(all_data_pdf.region)
all_data_pdf = all_data_pdf.query("region != 'крымскийфо'")
replacements = {
    'россия': 'российскаяфедерация',
    'гормосква': 'москва',
    'горсанктпетербург': 'санктпетербург',
    'городcевастополь': 'севастополь',
    'ингушскаяреспублика': 'республикаингушетия',
    'республикасевосетия': 'республикасевернаяосетияалания',
    'респсевернаяосетия': 'республикасевернаяосетияалания',
    'кабардинобалкарскаяресп': 'кабардинобалкарскаяреспублика',
    'респкабардинобалкария': 'кабардинобалкарскаяреспублика',
    'карачаевочеркесскаяресп': 'карачаевочеркесскаяреспублика',
    'респкарачаевочеркесия': 'карачаевочеркесскаяреспублика',
    'пермскаяобласть': 'пермскийкрай',
    'камчатскаяобласть': 'камчатскийкрай',
    'республикачечня': 'чеченскаяреспублика',
    'республикаудмуртия': 'удмуртскаяреспублика',
    'республикачувашия': 'чувашскаяреспублика',
    'архангельскаяоблбао': 'архангельскаяобластьбезавтономногоокруга',
    'тюменскаяоблбао': 'тюменскаяобластьбезавтономныхокругов',
    'тюменскаяоблбезао': 'тюменскаяобластьбезавтономныхокругов',
    'тюменскаяобластьсао': 'тюменскаяобласть',
    'хантымансийскийао': 'хантымансийскийавтономныйокругюгра',
    'фо': 'федеральныйокруг',
    'автокруг': 'автономныйокруг',
    'автобласть': 'автономнаяобласть',
    'автобл': 'автономнаяобласть',
    'ао': 'автономныйокруг'}
for key, value in replacements.items():
    all_data_pdf.region = all_data_pdf.region.str.replace(key, value)           
regions.loc[-1] = ['Читинская область', 76999999, 76999999, 'регион', 'читинскаяобласть']

In [11]:
# Добавляем данные по регионам и индикаторам
all_data = all_data_pdf.merge(regions, how='left', on='region').merge(indicators, how='left', on='indicator_short_code')
all_data.index = all_data_pdf.index

# Объединяем комментарии по нозологиям и индикаторам
all_data['comment'] = all_data.apply(lambda row: 
                    row['comment_x'] if pd.notna(row['comment_x']) and pd.isna(row['comment_y']) else 
                    row['comment_y'] if pd.isna(row['comment_x']) and pd.notna(row['comment_y']) else 
                    f"{row['comment_x']}; {row['comment_y']}" if pd.notna(row['comment_x']) and pd.notna(row['comment_y']) else 
                    '', 
                    axis=1)

# Наводим финальную красоту
all_data = all_data.drop(columns={'region','indicator_short_code', 'comment_x', 'comment_y'})

for col in ['indicator_section','indicator_name', 'indicator_unit', 'indicator_code',
            'nosology_name','nosology_code','soc_dem_group',
            'object_name', 'object_level', 'object_oktmo','object_okato','comment']:
    all_data[col] = all_data[col].astype('string')  
    
all_data = all_data[['indicator_section','indicator_name', 'indicator_unit', 'indicator_code',
                     'nosology_name','nosology_code','soc_dem_group',
                     'object_name', 'object_level', 'object_oktmo','object_okato',
                     'year','indicator_value','comment']]

In [12]:
# Выгружаем таблицу в csv
all_data.to_csv('data/processed/cancer_care.csv', sep=';', index=False)